In all versions 2.x the narrative gets cleaned as before and products are cleaned twice. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time the same happens with the six most occuring ones. This run uses 5 different principle components.

__Initialing the data__

In [2]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_both.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 3


__Initializing the bag of words__

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [4]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(208872, 5)


In [5]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.18578609 -3.61092799  0.13197781  0.78822846  0.12734307]
 [ 3.99741843 -1.19769356  0.72043814 -0.68818075  0.31232516]
 [ 2.08671505  0.90257542  2.2977584  -0.95105684 -0.47205625]
 ...
 [ 8.09025741 -8.10549452 -1.76294362 -6.57218189  8.90049378]
 [ 5.24479333 -5.01219469 -0.97876904 -2.41989919  3.06272885]
 [ 3.8196355  -2.15453383  0.58298244 -1.84334449  2.81534325]]
                0          1         2         3         4
0        4.185786  -3.610928  0.131978  0.788228  0.127343
1        3.997418  -1.197694  0.720438 -0.688181  0.312325
2        2.086715   0.902575  2.297758 -0.951057 -0.472056
3       24.306424 -15.933382  3.728391  8.876167 -9.848902
4        3.268770   0.855641  2.253067 -1.290437 -0.392561
...           ...        ...       ...       ...       ...
208867   6.923291  -4.363558  1.390731 -1.148501  1.043125
208868   8.448625  -3.973068 -0.027313 -5.175088  6.178807
208869   8.090257  -8.105495 -1.762944 -6.572182  8.900494
208870   5.244793  -5.0121

__One hot encoding of extra columns__

In [7]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(208872, 5)
(208872, 161)
(208872, 166)


__Split into train and test__

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(156654, 166)
(52218, 166)
(156654,)
(52218,)


__Application of linear SVM__

In [9]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3763
                                                 Checking or savings account       0.99      1.00      1.00      4713
                                                               Consumer Loan       0.95      0.76      0.85      2933
                                                                 Credit card       1.00      1.00      1.00      4680
                                                 Credit card or prepaid card       0.95      0.96      0.96      4657
                                                            Credit reporting       1.00      1.00      1.00      4739
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.86      0.92      6081
                                                       